In [1]:
import pandas as pd

In [2]:
# Check present working directory for filenames
import os
path = os.getcwd() 
dir_list = os.listdir(path) 
  
print("Files and directories in '", path, "' :")  
for item in dir_list:
    print(item)
  


Files and directories in ' C:\Users\vince\Desktop\lambdaschool_temp\4.portfolioprojects\eat_health ' :
.ipynb_checkpoints
archive
atususersguide.pdf
ehact_2006.dat
ehact_2007.dat
ehact_2008.dat
ehact_2014.dat
ehact_2015.dat
ehact_2016.dat
ehchild_2006.dat
ehchild_2007.dat
ehchild_2008.dat
ehmintcodebk1416.pdf
ehmintcodebk_06-08.pdf
ehresp_2006.dat
ehresp_2007.dat
ehresp_2008.dat
ehresp_2014.dat
ehresp_2015.dat
ehresp_2016.dat
read_clean.ipynb
stitch.ipynb


In [3]:
from IPython.display import display

df1 = pd.read_csv('ehact_2006.dat')
df2 = pd.read_csv('ehchild_2006.dat')
df3 = pd.read_csv('ehresp_2006.dat')


display(df1.head(2), df2.head(2), df3.head(2))

,TUCASEID,TUACTIVITY_N,EUEATSUM,EUDRKSUM,EUEDUR,EUDDUR,EUEDUR24,EUDDUR24
0,20060101060001,1,-1,-1,-1,-1,-1,-1
1,20060101060001,2,-1,-1,-1,-1,-1,-1


,TUCASEID,TULINENO,EXLCH,EELCH,EEBRK,EXBRK
0,20060101060014,2,0,2,2,0
1,20060101060014,3,0,1,2,0


,TUCASEID,TULINENO,EUFSP,EUGENHTH,EUHGT,EUINCOME2,ETWGT,ETHGT,EUINCLVL,EUFINLWGT,...,ERHHCH,ERBMI,EXSCLBRK,EESCLBRK,EXSCLLCH,EESCLLCH,ERPLWC,ERPBWC,EREATAD,ERDRKAD
0,20060101060001,1,2,3,66,2,0,0,1,3.378223e+06,...,3,25.0,-1,-1,-1,-1,-1.0,-1.0,-1,-1
1,20060101060014,1,2,5,67,1,0,0,1,3.498809e+06,...,3,36.0,0,2,0,1,0.5,0.0,-1,-1


In [4]:
display(df1.shape, df2.shape, df3.shape)

(146426, 8)

(7719, 6)

(12891, 33)

## Interpret/Transform: Eating & Health Activity File (EHACT - df1)
**Identify data to keep.  Drop per column name match criteria (see below)**

* TUCASEID: case_id
* TUACTIVITY_N: activity_number
* EUEATSUM: eating_during_activity
* EUDRKSUM: drinking_during_activity (non-water implied for analysis.  not included in new var name)
* EUEDUR: eating_duration
* EUDDUR: drinking_duration
* EUEDUR24: eating_duration_24h
* EUDDUR24: drinking_druation_24h

**Note, EHACT does not include demographic information needed for application**

For ease of client understanding and analysis (as well as applicationd development), EUEDUR24/EUDDUR24 will be used in lieu of EUEDUR and EUDDUR.

In [5]:
ehact = df1.drop(columns=['EUEDUR', 'EUDDUR']).copy()

In [6]:
ehact_columns = [
    'case_id', 'activity_number', 'eating_during_activity', 
    'drinking_during_activity', 'eating_duration_24', 'drinking_duration_24'
]
ehact.columns = ehact_columns
display(df1.drop(columns=['EUEDUR', 'EUDDUR']).head(2), ehact.head(2))

,TUCASEID,TUACTIVITY_N,EUEATSUM,EUDRKSUM,EUEDUR24,EUDDUR24
0,20060101060001,1,-1,-1,-1,-1
1,20060101060001,2,-1,-1,-1,-1


,case_id,activity_number,eating_during_activity,drinking_during_activity,eating_duration_24,drinking_duration_24
0,20060101060001,1,-1,-1,-1,-1
1,20060101060001,2,-1,-1,-1,-1


### Deal with missing values or unusable data for our use case

**Acceptable values:

* eating_during_activity or drinking_during_activity are (0,1)
* eating_duration_24, drinking_duration_24 (>1 & <1440)

In [7]:
import numpy as np

def allocation_check(x):
    '''
    Check allocation column for acceptable value and apply np.nan to missing data.
    ATUS allocation values:
        0 Value – no change
        10 Value to value
        11 Blank to value
        12 Don’t know to value
        13 Refused to value
        21 Blank to longitudinal value
        22 Don’t know to longitudinal value
        23 Refused to longitudinal value
        30 Value to allocated longitudinal value (unused)
        31 Blank to allocated longitudinal value (unused)
        32 Don’t know to allocated longitudinal value (unused)
        33 Refused to allocated longitudinal value (unused)
        40 Value to allocated value
        41 Blank to allocated value
        42 Don’t know to allocated value
        43 Refused to allocated value
    
    '''
    value_present_allocations = [0, 10, 11, 12, 13, 21, 22, 23, 30, 31, 32, 33,
                                 40, 41, 42, 43]
    if x in value_present_allocations:
        return 1
    else:
        return 0
    

In [8]:
import numpy as np

def value_check(x, minima, maxima, nan_opt):
    if x >= minima and x <= maxima:
        return x
    else:
        if nan_opt:
            return np.nan
        else:
            return 0

In [9]:
ehact.eating_duration_24.value_counts().head()

-1     136735
 10      2178
 5       2028
 15      1559
 30       903
Name: eating_duration_24, dtype: int64

In [10]:
# Convert -1 to 0 for multiplication to eating_duration_24
ehact['eating_during_activity'] = ehact.eating_during_activity.apply(value_check, args=(0,2,False))
ehact['drinking_during_activity'] = ehact.drinking_during_activity.apply(value_check, args=(0,2,False))

ehact.eating_during_activity.value_counts()

0    136735
1      9691
Name: eating_during_activity, dtype: int64

In [11]:
# Multiply binary existance (eating_during_activity) by duration to standardize missing values as 0.
# Then keep only positive numbers.
# Drop binary existance columns
ehact['eating_duration_24'] = ehact['eating_duration_24'] * ehact['eating_during_activity']
ehact['drinking_duration_24'] = ehact['drinking_duration_24'] * ehact['drinking_during_activity']

ehact = ehact.drop(columns=['eating_during_activity', 'drinking_during_activity'])

ehact['eating_duration_24'] = ehact['eating_duration_24'].apply(value_check, args=(1,1440, True))
ehact['drinking_duration_24'] = ehact['drinking_duration_24'].apply(value_check, args=(1,1440, True))

In [12]:
# Groupby case_id to create features secondary_eating_24, secondary_drinking_24
# INCOMPLETE

In [117]:
# 
ehact = ehact.set_index('case_id')

In [118]:
display(ehact.head(2), ehact.isna().sum())

,activity_number,eating_duration_24,drinking_duration_24
case_id,,,
20060101060001,1,NaN,NaN
20060101060001,2,NaN,NaN


activity_number              0
eating_duration_24      136793
drinking_duration_24    137079
dtype: int64

## Interpret/Transform: Eating & Health Child Responses

Analysis of the children directly is not intended.  The goal here is to profile the household and append to the eventual final table for modeling.

**Identify data to keep. Drop per column name match criteria (see below)**

* TUCASEID: case_id
* TULINENO: person_number
* EXLCH: ate_school_lunch_allocation
* EELCH: ate_school_lunch
* EEBRK: ate_school_breakfast
* EXBRK: ate_school_breakfast_allocation

### Engineered Features
**One row per case_id**

* total_children
* num_assisted_meals_children

In [87]:
ehchild_columns = [
    'case_id', 'person_number', 'ate_school_lunch_allocation',
    'ate_school_lunch', 'ate_school_breakfast', 'ate_school_breakfast_allocation' 
]
ehchild = df2.copy()
ehchild.columns = ehchild_columns
display(df2.head(2), ehchild.head(2))

,TUCASEID,TULINENO,EXLCH,EELCH,EEBRK,EXBRK
0,20060101060014,2,0,2,2,0
1,20060101060014,3,0,1,2,0


,case_id,person_number,ate_school_lunch_allocation,ate_school_lunch,ate_school_breakfast,ate_school_breakfast_allocation
0,20060101060014,2,0,2,2,0
1,20060101060014,3,0,1,2,0


In [89]:
# Check for valid allocation codes
ehchild['ate_school_lunch_allocation'] = ehchild['ate_school_lunch_allocation'].apply(allocation_check)

,case_id,person_number,ate_school_lunch_allocation,ate_school_lunch,ate_school_breakfast,ate_school_breakfast_allocation
count,7.719000e+03,7719.000000,7719.0,7719.000000,7719.000000,7719.000000
mean,2.006062e+13,3.664853,1.0,1.210001,1.706827,0.161290
std,3.748757e+08,1.401050,0.0,0.407336,0.455246,2.391306
min,2.006010e+13,1.000000,1.0,1.000000,1.000000,0.000000
25%,2.006030e+13,3.000000,1.0,1.000000,1.000000,0.000000
50%,2.006050e+13,4.000000,1.0,1.000000,2.000000,0.000000
75%,2.006101e+13,4.000000,1.0,1.000000,2.000000,0.000000
max,2.006121e+13,13.000000,1.0,2.000000,2.000000,41.000000


In [92]:
# Check for valid allocation codes
ehchild['ate_school_breakfast_allocation'] = ehchild['ate_school_breakfast_allocation'].apply(allocation_check)

In [97]:
# Set invalid data to zero where appropriate
ehchild['ate_school_lunch'] = ehchild.ate_school_lunch * ehchild.ate_school_lunch_allocation
ehchild['ate_school_breakfast'] = ehchild.ate_school_breakfast * ehchild.ate_school_breakfast_allocation

In [100]:
# Drop allocation columns
ehchild = ehchild.drop(columns=['ate_school_lunch_allocation', 'ate_school_breakfast_allocation'])

In [102]:
# Convert 2's in eating to zero, leave 1s as mean provided
ehchild['ate_school_breakfast'] = ehchild.ate_school_breakfast % 2
ehchild['ate_school_lunch'] = ehchild.ate_school_lunch % 2

In [107]:
# Set all person_numbers to 1 for simple summation to number children
ehchild['person_number'] = 1

In [126]:
# Group by case_id and sum meals and children in household.  Rename columns
ehchild_condensed = ehchild.groupby('case_id').sum()
ehchild_condensed['assisted_meals'] = ehchild_condensed.ate_school_breakfast + ehchild_condensed.ate_school_lunch
ehchild_condensed = ehchild_condensed.drop(columns=['ate_school_lunch', 'ate_school_breakfast'])
ehchild_condensed.columns = ['num_children', 'assisted_meals']
ehchild_condensed.head(10)

,num_children,assisted_meals
case_id,,
20060101060014,2,1
20060101060033,2,2
20060101060057,4,4
20060101060067,2,1
20060101060092,2,2
20060101060108,4,8
20060101060500,1,1
20060101060534,1,1
20060101060545,2,2


In [128]:
# Combine ehchild_condensed and ehact --> NOT WORKING CORRECTLY.  GROUPBY EHACT
pd.concat([ehact, ehchild_condensed], sort=False).head(100)

,activity_number,eating_duration_24,drinking_duration_24,num_children,assisted_meals
case_id,,,,,
20060101060001,1.0,NaN,NaN,NaN,NaN
20060101060001,2.0,NaN,NaN,NaN,NaN
20060101060001,3.0,NaN,NaN,NaN,NaN
20060101060001,4.0,NaN,NaN,NaN,NaN
20060101060001,5.0,NaN,NaN,NaN,NaN
20060101060001,6.0,NaN,NaN,NaN,NaN
20060101060001,7.0,NaN,NaN,NaN,NaN
20060101060001,8.0,NaN,NaN,NaN,NaN
20060101060001,9.0,NaN,NaN,NaN,NaN
